In [ ]:
import numpy as np
from keras.applications.densenet import DenseNet121
import keras.backend as K
import cv2
import os
from keras.callbacks import LearningRateScheduler,ReduceLROnPlateau,CSVLogger,ModelCheckpoint
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score,roc_auc_score
from sklearn.metrics import cohen_kappa_score,classification_report,auc,roc_curve
from keras.callbacks import Callback
from keras.layers import Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
# from tqdm import tqdm

In [ ]:
imagenet_mean = np.array([0.485, 0.456, 0.406])
imagenet_std = np.array([0.229, 0.224, 0.225])

def img_standardization(x):
    
#     x=cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x=x.astype('float16')/255.
#     return x
    return ((x-imagenet_mean)/imagenet_std)

In [ ]:
#Custom callback function to calculate average ROC,F1 and Kappa score
class roc_callback(Callback):
    def __init__(self,validation_data):
        self.X_val = validation_data[0]
        self.Y_val = validation_data[1]
        
#         X_val=np.zeros(self.x_val.shape,dtype='float32')
#         for index in range(self.x_val.shape[0]):
#             X_val[index]=img_standardization(self.x_val[index])
#         self.X_val=X_val

    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):

        if(epoch%1==0):
            print("Calc Roc")
            all_rocs = []
            y_pred_val = self.model.predict(self.X_val,verbose=1)

            try:
                roc_val = roc_auc_score(self.Y_val, y_pred_val)
                all_rocs.append(roc_val)
            except:
                pass

            all_rocs = np.array(all_rocs)
            mean_roc = np.mean(all_rocs)
            
            Y_val_kappa=[np.argmax(i) for i in self.Y_val]
            y_pred_kappa=[np.argmax(i) for i in y_pred_val]
                        
            avg_f1 = f1_score(Y_val_kappa, y_pred_kappa, average='weighted')
            kappa_score = cohen_kappa_score(Y_val_kappa, y_pred_kappa) 
            
            print("Mean ROC VAL {0}".format(mean_roc))
            print("AVG F1 {0}".format(avg_f1))
            print("KAPPA {0}".format(kappa_score))
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return


In [ ]:
from matplotlib import pyplot as plt
import math
from keras.callbacks import LambdaCallback
import keras.backend as K
import numpy as np


class LRFinder:
    """
    Plots the change of the loss function of a Keras model when the learning rate is exponentially increasing.
    See for details:
    https://towardsdatascience.com/estimating-optimal-learning-rate-for-a-deep-neural-network-ce32f2556ce0
    """

    def __init__(self, model):
        self.model = model
        self.losses = []
        self.lrs = []
        self.best_loss = 1e9

    def on_batch_end(self, batch, logs):
        # Log the learning rate
        lr = K.get_value(self.model.optimizer.lr)
        self.lrs.append(lr)

        # Log the loss
        loss = logs['loss']
        self.losses.append(loss)

        # Check whether the loss got too large or NaN
        if batch > 5 and (math.isnan(loss) or loss > self.best_loss * 4):
            self.model.stop_training = True
            return

        if loss < self.best_loss:
            self.best_loss = loss

        # Increase the learning rate for the next batch
        lr *= self.lr_mult
        K.set_value(self.model.optimizer.lr, lr)

    def find(self, x_train, y_train, start_lr, end_lr, batch_size=32, epochs=1):
        # If x_train contains data for multiple inputs, use length of the first input.
        # Assumption: the first element in the list is single input; NOT a list of inputs.
        N = x_train[0].shape[0] if isinstance(x_train, list) else x_train.shape[0]

        # Compute number of batches and LR multiplier
        num_batches = epochs * N / batch_size
        self.lr_mult = (float(end_lr) / float(start_lr)) ** (float(1) / float(num_batches))
        # Save weights into a file
        self.model.save_weights('tmp.h5')

        # Remember the original learning rate
        original_lr = K.get_value(self.model.optimizer.lr)

        # Set the initial learning rate
        K.set_value(self.model.optimizer.lr, start_lr)

        callback = LambdaCallback(on_batch_end=lambda batch, logs: self.on_batch_end(batch, logs))

        self.model.fit(x_train, y_train,
                       batch_size=batch_size, epochs=epochs,
                       callbacks=[callback], verbose=1)

        # Restore the weights to the state before model fitting
        self.model.load_weights('tmp.h5')

        # Restore the original learning rate
        K.set_value(self.model.optimizer.lr, original_lr)

    def find_generator(self, generator, start_lr, end_lr, epochs=1, steps_per_epoch=None, **kw_fit):
        if steps_per_epoch is None:
            try:
                steps_per_epoch = len(generator)
            except (ValueError, NotImplementedError) as e:
                raise e('`steps_per_epoch=None` is only valid for a'
                        ' generator based on the '
                        '`keras.utils.Sequence`'
                        ' class. Please specify `steps_per_epoch` '
                        'or use the `keras.utils.Sequence` class.')
        self.lr_mult = (float(end_lr) / float(start_lr)) ** (float(1) / float(epochs * steps_per_epoch))

        # Save weights into a file
        self.model.save_weights('tmp.h5')

        # Remember the original learning rate
        original_lr = K.get_value(self.model.optimizer.lr)

        # Set the initial learning rate
        K.set_value(self.model.optimizer.lr, start_lr)

        callback = LambdaCallback(on_batch_end=lambda batch,
                                                      logs: self.on_batch_end(batch, logs))

        self.model.fit_generator(generator=generator,
                                 epochs=epochs,
                                 steps_per_epoch=steps_per_epoch,
                                 callbacks=[callback], verbose=1,
                                 **kw_fit)

        # Restore the weights to the state before model fitting
        self.model.load_weights('tmp.h5')

        # Restore the original learning rate
        K.set_value(self.model.optimizer.lr, original_lr)

    def plot_loss(self, n_skip_beginning=10, n_skip_end=5, x_scale='log'):
        """
        Plots the loss.
        Parameters:
            n_skip_beginning - number of batches to skip on the left.
            n_skip_end - number of batches to skip on the right.
        """
        plt.ylabel("loss")
        plt.xlabel("learning rate (log scale)")
        plt.plot(self.lrs[n_skip_beginning:-n_skip_end], self.losses[n_skip_beginning:-n_skip_end])
        plt.xscale(x_scale)
        plt.show()

    def plot_loss_change(self, sma=1, n_skip_beginning=10, n_skip_end=5, y_lim=(-0.01, 0.01)):
        """
        Plots rate of change of the loss function.
        Parameters:
            sma - number of batches for simple moving average to smooth out the curve.
            n_skip_beginning - number of batches to skip on the left.
            n_skip_end - number of batches to skip on the right.
            y_lim - limits for the y axis.
        """
        derivatives = self.get_derivatives(sma)[n_skip_beginning:-n_skip_end]
        lrs = self.lrs[n_skip_beginning:-n_skip_end]
        plt.ylabel("rate of loss change")
        plt.xlabel("learning rate (log scale)")
        plt.plot(lrs, derivatives)
        plt.xscale('log')
        plt.ylim(y_lim)
        plt.show()

    def get_derivatives(self, sma):
        assert sma >= 1
        derivatives = [0] * sma
        for i in range(sma, len(self.lrs)):
            derivatives.append((self.losses[i] - self.losses[i - sma]) / sma)
        return derivatives

    def get_best_lr(self, sma, n_skip_beginning=10, n_skip_end=5):
        derivatives = self.get_derivatives(sma)
        best_der_idx = np.argmax(derivatives[n_skip_beginning:-n_skip_end])[0]
        return self.lrs[n_skip_beginning:-n_skip_end][best_der_idx]

In [ ]:
from keras.callbacks import *

class CyclicLR(Callback):
    """This callback implements a cyclical learning rate policy (CLR).
    The method cycles the learning rate between two boundaries with
    some constant frequency, as detailed in this paper (https://arxiv.org/abs/1506.01186).
    The amplitude of the cycle can be scaled on a per-iteration or 
    per-cycle basis.
    This class has three built-in policies, as put forth in the paper.
    "triangular":
        A basic triangular cycle w/ no amplitude scaling.
    "triangular2":
        A basic triangular cycle that scales initial amplitude by half each cycle.
    "exp_range":
        A cycle that scales initial amplitude by gamma**(cycle iterations) at each 
        cycle iteration.
    For more detail, please see paper.
    
    # Example
        ```python
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., mode='triangular')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```
    
    Class also supports custom scaling functions:
        ```python
            clr_fn = lambda x: 0.5*(1+np.sin(x*np.pi/2.))
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., scale_fn=clr_fn,
                                scale_mode='cycle')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```    
    # Arguments
        base_lr: initial learning rate which is the
            lower boundary in the cycle.
        max_lr: upper boundary in the cycle. Functionally,
            it defines the cycle amplitude (max_lr - base_lr).
            The lr at any cycle is the sum of base_lr
            and some scaling of the amplitude; therefore 
            max_lr may not actually be reached depending on
            scaling function.
        step_size: number of training iterations per
            half cycle. Authors suggest setting step_size
            2-8 x training iterations in epoch.
        mode: one of {triangular, triangular2, exp_range}.
            Default 'triangular'.
            Values correspond to policies detailed above.
            If scale_fn is not None, this argument is ignored.
        gamma: constant in 'exp_range' scaling function:
            gamma**(cycle iterations)
        scale_fn: Custom scaling policy defined by a single
            argument lambda function, where 
            0 <= scale_fn(x) <= 1 for all x >= 0.
            mode paramater is ignored 
        scale_mode: {'cycle', 'iterations'}.
            Defines whether scale_fn is evaluated on 
            cycle number or cycle iterations (training
            iterations since start of cycle). Default is 'cycle'.
    """

    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())        
            
    def on_batch_end(self, epoch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr())

In [ ]:
def batch_generator(X, Y, batch_size = 32):
    indices = np.arange(len(X))
    batch=[]
    while True:
            # it might be a good idea to shuffle your data before each epoch
            np.random.shuffle(indices)
            for i in indices:
                batch.append(i)
                if len(batch)==batch_size:
                    x=img_standardization(X[batch])
                    yield x, Y[batch]
                    batch=[]

In [ ]:
X_val=np.zeros(x_val.shape,dtype='float32')
for index in range(x_val.shape[0]):
    X_val[index]=img_standardization(x_val[index])
    print (index)

In [ ]:
Y_train=[]
for i in y_train:
    if i.any()!=1:
        Y_train.append(0)
    else:
        Y_train.append(1)
Y_train=to_categorical(Y_train,num_classes=2)

Y_val=[]
for i in y_val:
    if i.any()!=1:
        Y_val.append(0)
    else:
        Y_val.append(1)
Y_val=to_categorical(Y_val,num_classes=2)

roc_call = roc_callback((X_val,Y_val))

#Conversion of labels into One-hot encoding
Y_train[np.where(Y_train==2)]=1
Y_val[np.where(Y_val==2)]=1

Y_train=to_categorical(Y_train,num_classes=2)
Y_val=to_categorical(Y_val,num_classes=2)

roc_call = roc_callback((X_val,Y_val))


In [ ]:
print (X_train.shape)
print (Y_train.shape)
print (X_val.shape)
print (Y_val.shape)

In [ ]:
from keras.optimizers import Adam
adam=Adam(lr=1e-4)

#Model declaration:Densenet 
dense_model = DenseNet121(weights='imagenet', include_top=False,input_shape=(224,224,3),pooling='avg')
preds = Dense(14,activation='sigmoid')(dense_model.output)
model = Model(dense_model.input,preds)
model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])
model.load_weights('/opt/bucketdata/Users/Rohit/15_pathology_model/NIH_more_than_decent_weights/updated_best_weights.h5')


#Model parameters for compiling

model.compile(optimizer=adam,loss='binary_crossentropy',metrics=['accuracy'])


In [ ]:
new_preds = Dense(2,activation='softmax')(model.get_layer('avg_pool').output)
new_model = Model(model.input,new_preds)
new_model.compile(optimizer=adam,loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
from sklearn.utils import class_weight
y_true=np.argmax(Y_train,axis=-1)
weights = class_weight.compute_class_weight('balanced',np.unique(y_true),y_true)
print(weights)

new_model.load_weights("./Weights/CHAI_PvsNP_newlabels_transferlearned_fromNIHPvsNP.hdf5")

In [ ]:
lr_finder = LRFinder(new_model)

In [ ]:
lr_finder.find_generator(batch_generator(X_train, Y_train), start_lr=1e-6, end_lr=1e-3, steps_per_epoch=len(X_train)/32, epochs=5)

In [ ]:
lr_finder.plot_loss(n_skip_beginning=5, n_skip_end=5)

In [ ]:
lr_finder.plot_loss_change(sma=20, n_skip_beginning=5, n_skip_end=5, y_lim=(-0.01, 0.01))

In [ ]:
# import the necessary packages
import os

# initialize the list of class label names
CLASSES = ["Healthy", "Pathology"]

# define the minimum learning rate, maximum learning rate, batch size,
# step size, CLR method, and number of epochs
MIN_LR = 1e-6
MAX_LR = 1e-3
BATCH_SIZE = 32
STEP_SIZE = 4
CLR_METHOD = "exp_range" #triangular2, triangular
NUM_EPOCHS = 100

# define the path to the output training history plot and cyclical
# learning rate plot
#TRAINING_PLOT_PATH = os.path.sep.join(["output", "training_plot.png"])
#CLR_PLOT_PATH = os.path.sep.join(["output", "clr_plot.png"])


clr = CyclicLR(
	mode=CLR_METHOD,
	base_lr=MIN_LR,
	max_lr=MAX_LR,
	step_size= STEP_SIZE * (X_train.shape[0] // BATCH_SIZE))

In [ ]:
#Callback for saving model weights based on minimum Validation loss 
model_checkpoint = ModelCheckpoint(
        os.path.join('./Weights/', 'NIH_PvsNP_19022020_clr.hdf5'),
        monitor='val_loss', mode='min',save_best_only=True, verbose=1, save_weights_only=True)

#Callback for reducing learning rate based on validation loss
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,patience=5, min_delta=0.0001, verbose=1, min_lr=0.000001)

#Callback for storing logs of learning rate, loss and accuracy
csvlogger=CSVLogger('NIH_PvsNP_19022020.csv')

callbacks = [model_checkpoint, roc_call,csvlogger, clr]


In [ ]:
# model.fit_generator(batch_generator_train(X_1, Y_train, batch_size=32),validation_data=batch_generator_val(X_val, Y_val, batch_size=32),steps_per_epoch=len(X_1) / 32, epochs=10,callbacks=callbacks,verbose=1,validation_steps=len(X_val) / 32)
new_model.fit_generator(batch_generator(X_train, Y_train, batch_size=32),validation_data=(X_val, Y_val),steps_per_epoch=len(X_train) / 32, epochs=100,callbacks=callbacks,verbose=1, shuffle=True, class_weight=weights)

new_model.load_weights("/opt/bucketdata/Users/gunjan/CHAI/tb_data_experiments/other_analysis/model_creation/hp/data_generator/py_files/combined_data/(nih+CHAI)>CHAI/CHAI_hp_adam1e-6_denseblock_freezed.hdf5")

In [ ]:
new_model.load_weights("./Weights/NIH_PvsNP_19022020_clr.hdf5")

In [ ]:
X_val.shape

In [ ]:
# X_test=[]
# for index in range(X_val.shape[0]):
#     X_test.append(img_standardization(X_val[index]))
# X_test=np.array(X_test)

op=new_model.predict(X_val,verbose=1)
# op=model.predict_generator(batch_generator_val(x_val, Y_val, batch_size=1),steps=len(X_val) /1,verbose=1)

In [ ]:
Y_pred=[]
for i in op:
    if i[1]>=0.5:
        Y_pred.append(1)
    else:
        Y_pred.append(0)

In [ ]:
Y_pred=np.argmax(op,axis=1)
Y_test=np.argmax(Y_val,axis=-1)
# Y_test=Y_val.copy()
target_names = ['normal','Pathology']

print("Confusion Matrix:")
print (confusion_matrix(Y_test,Y_pred))

print("Classification report:")
print(classification_report(Y_test,Y_pred,target_names=target_names))

print("Kappa score:")
print(cohen_kappa_score(Y_test,Y_pred))

# Converting Healthy+other into NonTB cases to calculate ROC
y_true=np.array(Y_test)
y_true[np.where(y_true==1)]=1
y_true[np.where(y_true==2)]=1

#Taking probabilities of TB class
preds = op[:,1]
fpr, tpr, threshold = roc_curve(y_true, preds)
roc_auc = auc(fpr, tpr)


# ROC curve plot
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.savefig('TB_ROC_curve.png',dpi=200)
plt.show()



In [ ]:
op.shape